# Marker genes for eye data from the Cell Census

The Cell Census is a versioned container for the single-cell data hosted at [CELLxGENE Discover](https://cellxgene.cziscience.com/). The Cell Census utilizes [SOMA](https://github.com/single-cell-data/SOMA/blob/main/abstract_specification.md) powered by [TileDB](https://tiledb.com/products/tiledb-embedded) for storing, accessing, and efficiently filtering data.

TBD

# Contents
TBD

## Opening the census

The `cell_census` python package contains a convenient API to open the latest version of the Cell Census.

In [1]:
import cell_census
import numpy as np
import memento
import pandas as pd

census = cell_census.open_soma()

You can learn more about the `cell_census` methods by accessing their corresponding documentation via `help()`. For example `help(cell_census.open_soma)`.

## Load data

Getting dataset of interest

In [2]:
# Tabula muris senis 10x liver
dataset_title = "All major cell types in adult human retina"

# Getting the joinid and dataset id
datasets = census["census_info"]["datasets"].read(value_filter = f"dataset_title == '{dataset_title}'").concat().to_pandas()
dataset_joinid = list(datasets["soma_joinid"])
datasset_interest_id = datasets["dataset_id"][0]

Get measured genes in dataset

In [3]:
presence_matrix = cell_census.get_presence_matrix(census, organism="Homo sapiens", measurement_name="RNA")
dataset_presnence = presence_matrix[dataset_joinid,]

gene_joinid = np.nonzero(dataset_presnence.sum(axis=0).A1 == dataset_presnence.shape[0])[0].tolist()

Get anndata

In [4]:
assays = ["10x 3' v3", "10x 3' v2"]
adata = cell_census.get_anndata(
    census=census,
    organism="Homo sapiens",
    var_coords= gene_joinid,
    #obs_value_filter=f"dataset_id == '{datasset_interest_id}'",
    obs_value_filter=f"tissue_general == 'eye' and assay in {assays}",
)

In [5]:
adata.var_names = adata.var["feature_name"]
adata.var

,soma_joinid,feature_id,feature_name,feature_length
feature_name,,,,
RP11-34P13.7,0,ENSG00000238009,RP11-34P13.7,3726
AP006222.1,2,ENSG00000228463,AP006222.1,8224
RP4-669L17.4,3,ENSG00000237094,RP4-669L17.4,6204
RP11-206L10.17,4,ENSG00000230021,RP11-206L10.17,5495
LINC01409,5,ENSG00000237491,LINC01409,8413
...,...,...,...,...
RP11-398A8.4,52844,ENSG00000273461,RP11-398A8.4,496
LL09NC01-139C3.1,52846,ENSG00000273473,LL09NC01-139C3.1,572
RP11-108L7.14,52847,ENSG00000273476,RP11-108L7.14,448


In [6]:
aaa = adata.obs["cell_type"].value_counts()
list(zip(list(aaa.index), list(aaa)))

[('retinal rod cell', 151112),
 ('amacrine cell', 122396),
 ('retinal bipolar neuron', 56261),
 ('Mueller cell', 50545),
 ('ON-bipolar cell', 46059),
 ('OFF-bipolar cell', 41015),
 ('fibroblast', 38487),
 ('corneal epithelial cell', 31410),
 ('retinal ganglion cell', 30729),
 ('retinal cone cell', 28626),
 ('melanocyte', 27699),
 ('pigmented epithelial cell', 27000),
 ('ciliary muscle cell', 21468),
 ('retina horizontal cell', 19603),
 ('glial cell', 18011),
 ('basal cell', 17465),
 ('pigmented ciliary epithelial cell', 16178),
 ('conjunctival epithelial cell', 14439),
 ('Schwann cell', 10765),
 ('corneal endothelial cell', 8394),
 ('macrophage', 7191),
 ('non-pigmented ciliary epithelial cell', 6327),
 ('anterior lens cell', 6160),
 ('glycinergic neuron', 6065),
 ('macroglial cell', 5276),
 ('lens epithelial cell', 5276),
 ('rod bipolar cell', 5207),
 ('endothelial cell', 4872),
 ('native cell', 3704),
 ('smooth muscle cell of sphincter of pupil', 2221),
 ('astrocyte', 2031),
 ('endot

## Find Marker Genes (SCRAN-inspired) for ALL cell types

Let's do all cell types vs each other cell type. First perform all comparions across other cell types.

In [ ]:
all_markers = {}
all_cell_types = adata.obs.cell_type.drop_duplicates()
total_cell_types = len(all_cell_types)
n = 1
for cell_type_query in all_cell_types:
    print("Current cell type", n, ":", cell_type_query)
    current_markers = pd.DataFrame()
    for cell_type_target in adata.obs.cell_type.drop_duplicates():

        if cell_type_target == cell_type_query:
            continue

        # Set groupings
        adata.obs[cell_type_target] = adata.obs.cell_type == cell_type_query
        adata.obs[cell_type_target] = adata.obs[cell_type_target] * 1

        # Run differential gene expression
        differential_exp_results = memento.binary_test_1d(
            adata=adata[adata.obs.cell_type.isin([cell_type_target, cell_type_query]),], 
            capture_rate=0.07, 
            treatment_col=cell_type_target, 
            num_cpus=90,
            num_boot=5000
        )

        # Retain columns of interest and rename them for future joining
        differential_exp_results = differential_exp_results[["gene", "de_coef", "de_pval", "de_se"]]

        if current_markers.shape == (0,0):
            current_markers = differential_exp_results
        else:
            current_markers = current_markers.append(differential_exp_results, ignore_index=True)
    
    # Calculate the 0.1 percentile for each gene across comparisons
    current_markers = current_markers.groupby("gene").agg(lambda x: np.quantile(x, 0.1))
    current_markers["cell_type"] = cell_type_query
    all_markers[cell_type_query] = current_markers

In [40]:
all_markers_consolidated = pd.concat(all_markers.values(), ignore_index = False )

In [59]:
all_markers_consolidated.sort_values("de_coef", ascending=False).head(25)
cell_type = 'retinal pigment epithelial cell'
top_25 = all_markers_consolidated.query(f"cell_type == '{cell_type}'").sort_values("de_coef", ascending=False).head(25)
print(*list(top_25.index), sep = ", ")

TTR, RPE65, SLC16A8, BEST1, RBP1, SLC22A8, PRDM16-DT, TMEM98, LINC00276, INPP5K, SERPINF1, SLC16A14, LRAT, SFRP5, LINC02159, RDH5, CTSD, ITGB8, SLC16A3, KCNJ13, PITPNA, CSPG5, PTGDS, IFITM10, TPRN


In [58]:
all_markers_consolidated.sort_values("de_coef", ascending=False).head(25)

,de_coef,de_pval,de_se,cell_type
gene,,,,
HBB,8.959019,5.812335e-10,0.092343,erythroid lineage cell
HBA2,8.554856,4.809925e-10,0.114799,erythroid lineage cell
HBA1,7.776886,1.687857e-08,0.124033,erythroid lineage cell
MUC5AC,6.848498,2.045492e-05,0.284006,goblet cell
DGAT2,5.930711,6.788103e-05,0.353516,fat cell
SCGB2A1,5.532811,3.970476e-08,0.193530,stromal cell
LACRT,5.506518,2.609889e-08,0.176430,stromal cell
RP11-554D15.3,5.460024,1.318281e-07,0.087348,lens fiber cell
RP11-434D9.1,5.353935,3.755857e-11,0.023825,retinal cone cell
